In [2]:
from train import train
from markup import markup
from apply import integrate_image_signal_over_channels
import warnings
warnings.filterwarnings("ignore")

## Train part (if necessary)

In [3]:
unet = train(coco_json_path='./jsons/coco_all_labels.json', 
             train_images_path='./train_images/', target_channel_names='dapi,lipid', train_directory='.',
          validate=True, device='cpu', num_epochs=10, model_name='unet', init_features=10)

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Epoch number: 0
validation iou is 0.1973319005219527
Training epoch loss: 0.7590512037277222
Epoch 1 of 10 took 2.536s
Epoch number: 1
validation iou is 0.26608075378373686
Training epoch loss: 0.6880590915679932
Epoch 2 of 10 took 2.285s
Epoch number: 2
validation iou is 0.32235346268114046
Training epoch loss: 0.648160994052887
Epoch 3 of 10 took 2.232s
Epoch number: 3
validation iou is 0.3403956434062933
Training epoch loss: 0.5630679726600647
Epoch 4 of 10 took 2.135s
Epoch number: 4
validation iou is 0.3423439025158223
Training epoch loss: 0.6876410245895386
Epoch 5 of 10 took 2.205s
Epoch number: 5
validation iou is 0.3651903848096152
Training epoch loss: 0.54068922996521
Epoch 6 of 10 took 2.255s
Epoch number: 6
validation iou is 0.3828653047394832
Training epoch loss: 0.7104802131652832
Epoch 7 of 10 took 2.133s
Epoch number: 7
validation iou is 0.4285671026112937
Training epoch loss: 0.4296751022338867
Epoch 8 of 10 took 2.179s
Epoch number: 8
validation iou is 0.4486502779354

## Load model part 

In [3]:
import torch
from Unet import UNet

unet = UNet(in_channels=2, init_features=10)
unet.load_state_dict(torch.load('./models/unet_60_epochs_lipids_dapi'))

<All keys matched successfully>

## Markup of validation images


In [4]:
markup(model=unet, markup_images_path='val_images/', 
       target_channel_names='dapi,lipid', 
       save_path='./markuped_vals/', if_binary_mask=True)

## Calculate integrated signal from the cells' nucleis in each chanel

In [5]:
integrated_signal_dict = integrate_image_signal_over_channels(masks_directory_path='markuped_vals/',
                                                              images_path='val_images/',
                                                              target_channel_names='dapi,lipid',
                                                              interest_mask='dapi_0_0.tif', save=False)
                                                              

In [6]:
integrated_signal_dict

{'dapi': {'dapi_0_0.tif': 1107947}, 'lipid': {'lipid_0_0.tif': 582395}}

In [7]:
#Sanity Check
import skimage 

mask = skimage.io.imread('markuped_vals/dapi_0_0.tif')
image = skimage.io.imread('val_images/dapi/dapi_0_0.tif')
integrated_signal = (mask * image).sum()
print(f'Integrated signal={integrated_signal}')

Integrated signal=1107947.0
